# Acknowledgment:
This homework is adapted from Chris Manning and Dan Jurafsky's Coursera NLP class from 2012.

The goal of this assignment is to write regular expressions that extract phone numbers and regular expressions that extract email addresses from multiple of files.


## Examples of these emails and phones

    - jurafsky(at)cs.stanford.edu
    - jurafsky at csli dot stanford dot edu
    - <script type="text/javascript">obfuscate('stanford.edu','jurafsky')</script> 
    - TEL +1-650-723-0293
    - <a href="contact.html">TEL</a> +1&thinsp;650&thinsp;723&thinsp;0293
### Screen shot of one file as html
![Image](html_im.png)

## Solution
our solution to extract most of mails and phone numbers from all of files depends on the use of Regular Expression.


![Image2](solution.png)

In [1]:
import sys
import os
import re
import pprint
import pandas as pd

In [2]:
# Handling email expression
first_regular  = '(\w+\s?([.]\w+)?(at|WHERE|@|&#x40;)\s?\w+\.?( DOM | dot |;)?\w+\.?( dot | DOM | dt|;)?\w+)'
second_regular = '(([\w\.-]+)\s?(@|%20at%20| at | where )\s?(stanford|-s-t-a-n-f-o-r-d-|gradiance)(\.|%20dot%20| dot | dom | dt )(edu|-e-d-u|com))'
third_regular  = '(([\w\.]+)\s?\(followed? ?by? ?(\"|&ldquo;)@(cs\.)?stanford.edu?(\"|&rdquo;)\))'
fourth_regular = '((obfuscate|pal) ?(\(\')?\w+\.? ?\w+(\',\')? ?\w+ ?\w+(\'\);)?)'


# Handling phone expression
fifth_regular = '[\d]{3}-[\d]{3}-[\d]{4}'
six_regular   = '[(][\d]{3}[)] ?[\d]{3}-[\d]{4}'
seven_regular = '[+][0-9] ?[(]?[\d]{3}[)]? [\d]{3}[-| ][\d]{4}'



In [3]:
def to_lower(text):
    '''
        This function handle cases of upper and lower case
    '''
    return text.lower()

In [4]:
def clean_number(number):
    '''
        A function used to clean the numbers to be in form of 123-123-1234
        it is take a number in string way convert it to be a list
        then work on and return as string using join built-in function
    '''
    number = list(number)
    number.reverse()
    strs  = ""
    c_regular = ')- (+'
    for indx, c in enumerate(number):
        if c in c_regular:
            continue 
        if int(number[indx]) >=0 and int(number[indx]) <= 9:
            strs = strs + number[indx]
        if len(strs) == 4 or len(strs) == 8:
            strs +='-'
        if len(strs) == 12:
            break
    return strs[::-1]


In [5]:
def common_cases(m):
    '''
        This function is helful to dealing with some casses of has more spaces or taps between  words of mail
        like abdo   @ yahoo.
        com
        also some cases has a ; at end of the word like .edu; others like - or "
        other wise we git the char of th word
        return string at end of the function
    '''
    ret = ''
    c_reqular = ' \t\n-% "-)('
    for indx, c in enumerate(m):
        if c in c_reqular:
            ret+=''
        elif c == ';':
            ret +='.'
        else:
            ret +=c
    return ret

In [6]:

def clean_at_again(text):
    '''
        this function is general handle of different cases of words like dot, 
        DOM  and other like &#x40; and others
        return at the to another function that handle spaces and small things
    '''
    if "%20at%20" in text:
        text = text.split('%20')
    elif ' at ' in text:
        text = text.split(' ')
    elif ' WHERE ' in text:
        text = text.split(' ')
    elif ' WHERE ' in text:
        text = text.split(' ')
    elif '-@-' in text:
        text = text.split('-')
    elif '&#x40;' in text:
        text = text.partition('&#x40;')
    '''
        a partition is a function that search for something 
        and return with a tuple of 3 element 
        1 - everything before the "match" 
        2 - the "match" 
        3 - everything after the "match"
    '''
    ret = ''
    for word in text:
        if word == "dot" or word == "dt" or word == "DOM":
            ret +='.'
        elif word == "at" or word == "WHERE" or word == "&#x40;":
            ret +='@'
        else: ret +=str(word)
    return common_cases(ret)

In [7]:
def handle_stanford_without_dots(text):
    '''
        this function handle some cases of stanfordedu to be stanford.edu and cs
        so cases need to add . to be as formed e-mail we saw in general
    '''
    if 'cs' in text:
        if not 'cs.' in text:
            text = text.replace('cs', 'cs.')
    if 'stanford' in text:
        if not 'stanford.' in text:
            text = text.replace('stanford', 'stanford.')
    return text


In [8]:
def followed_by_handle(text):
    '''
        this function handle some cases of differnt cases like below we need to handle it
        to be as a genral mail at the end
    '''
    text = common_cases(text)
    if "followedby" in text or "&ldquo." in text  or "&rdquo." in text or 'obfuscate\'' in text:
        text = text.replace("followedby", '')
        text = text.replace("&ldquo.", '')
        text = text.replace("&rdquo.", '')
        text = text.replace("obfuscate\'", '')
        text.replace('at', '@')
    if 'jurafsky' in text:
        text = 'jurafsky@' + text.replace('jurafsky', 'stanford.edu')
    if 'at' in text:
        text = text.replace('at', '@')
    text = text.partition('\'')
    text = text[0]
    text = handle_stanford_without_dots(text)
    return text



In [9]:
def process_file(name, f):
    res = []
    for line in f:
        line = to_lower(line)
        matches = re.findall(first_regular,line)
        for m in matches:
            m = clean_at_again(m[0])
            if len(m) > 0 and '.' in m and m.count('@') == 1:
                res.append((name, 'e', m))

        matches = re.findall(second_regular,line)
        for m in matches:
            m = clean_at_again(m[0])
            res.append((name, 'e', m))

        matches = re.findall(third_regular,line)
        for m in matches:
            m = followed_by_handle(m[0])
            res.append((name, 'e', m))

        matches = re.findall(fourth_regular,line)
        for m in matches:
            m = followed_by_handle(m[0])
            if len(m) > 0 and '.' in m and m.count('@') == 1:
                res.append((name, 'e', m))
        '''
            numbers handling process
        '''
        matches = re.findall(fifth_regular,line)
        matches.extend(re.findall(six_regular,line))
        matches.extend(re.findall(seven_regular,line))
        for m in matches:
            m = clean_number(m)
            res.append((name,'p',m))
    return res


In [10]:
def process_dir(data_path):
    guess_list = []
    for fname in os.listdir(data_path):
        if fname[0] == '.':
            continue
        path = os.path.join(data_path,fname)
        f = open(path,'r', encoding='utf-8', errors='ignore')
        f_guesses = process_file(fname, f)
        guess_list.extend(f_guesses)
    emails_and_numbers = set(guess_list) # to remove duplicates and sort them
    return emails_and_numbers

In [22]:
# get the direction of dev folder that contain your files
dir_f = !pwd
dir_f = dir_f[0] + '/data/dev/'

In [18]:
emails_and_numbers = process_dir(dir_f) # to remove duplicates

In [27]:
print("Extracted emails and number from all of files in Dev set of 46 files\n\n")
emails_and_numbers

Our Extract emails and number from all of files in Dev set




{('ashishg', 'e', 'ashishg@stanford.edu'),
 ('ashishg', 'e', 'rozm@stanford.edu'),
 ('ashishg', 'p', '650-723-1614'),
 ('ashishg', 'p', '650-723-4173'),
 ('ashishg', 'p', '650-814-1478'),
 ('balaji', 'e', 'balaji@stanford.edu'),
 ('bgirod', 'p', '650-723-4539'),
 ('bgirod', 'p', '650-724-3648'),
 ('bgirod', 'p', '650-724-6354'),
 ('cheriton', 'e', 'cheriton@cs.stanford.edu'),
 ('cheriton', 'e', 'uma@cs.stanford.edu'),
 ('cheriton', 'p', '650-723-1131'),
 ('cheriton', 'p', '650-725-3726'),
 ('dabo', 'e', 'dabo@cs.stanford.edu'),
 ('dabo', 'p', '650-725-3897'),
 ('dabo', 'p', '650-725-4671'),
 ('dlwh', 'e', 'dlwh@stanford.edu'),
 ('engler', 'e', 'engler@lcs.mit.edu'),
 ('engler', 'e', 'englerwherestanforddomedu'),
 ('eroberts', 'e', 'eroberts@cs.stanford.edu'),
 ('eroberts', 'p', '650-723-3642'),
 ('eroberts', 'p', '650-723-6092'),
 ('fedkiw', 'e', 'fedkiw@cs.stanford.edu'),
 ('hager', 'e', 'hager@cs.jhu.edu'),
 ('hager', 'p', '410-516-5521'),
 ('hager', 'p', '410-516-5553'),
 ('hager', 